In [ ]:
# Using this repo in google colab: https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/5-Fine%20Tuning/5_3_QLoRA_Tuning.ipynb

# Load the PEFT and Datasets Libraries

In [1]:
!pip install -q accelerate==0.29.3
!pip install -q bitsandbytes==0.43.1
!pip install -q trl==0.8.6
!pip install -q peft==0.10.0
!pip install -q transformers==4.40.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does no

In [2]:
# Install the lastest versions of peft & transformers library recommended
# if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
# Import necessary classes
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer
import torch

# Hugging Face login

In [4]:
# Get huggingface token
from google.colab import userdata
HF_TOKEN=userdata.get('hf')
!huggingface-cli login --token $HF_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Load Model

In [5]:
model_name = "ajibawa-2023/Young-Children-Storyteller-Mistral-7B"

In [6]:
# Set quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [7]:
device_map = {"": 0}
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map=device_map,
                    use_cache = False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

# Original Model


In [23]:
# This function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=100):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        repetition_penalty=1.5, #Avoid repetition.
        early_stopping=True, #The model can stop before reach the max_length
        eos_token_id=tokenizer.eos_token_id,
    )
    return outputs

In [25]:
# Inference original model
input_sentences = tokenizer("Write a short bedtime story for children. Use simple words.",
                            return_tensors="pt").to('cuda')

foundational_outputs_sentence = get_outputs(foundation_model,
                                            input_sentences,
                                            max_new_tokens=1000,
                                            )

print(tokenizer.batch_decode(foundational_outputs_sentence,
                             skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Write a short bedtime story for children. Use simple words. [Note: This was translated from French.]\nOnce upon time, long ago in the land of animals who had never heard about daylight or nightfall before they were not allowed to have parties and play games because their parents said that it\'s "not good" during sunrise hours when many people wake up early morning calls going strong nearby farms making sounds like cockerels crowing away happily enjoying themselves! But one fine evening after twilight took its place by gently falling down over hillsides illuminating shadows casted all around town square where kids learned how some days can bring darkness too which wasn’t scary but adventurous instead – little mice whisper on grassy plain whilst watching stars above blink back at them; hedgehogs huddled together safe under thick thorn branches sharing stories while squirrel gathered nuts preparing lunches ahead knowing well enough those willpower moments spent wisely could lead him far

Once upon time, long ago in the land of animals who had never heard about daylight or nightfall before they were not allowed to have parties and play games because their parents said that it\'s "not good" during sunrise hours when many people wake up early morning calls going strong nearby farms making sounds like cockerels crowing away happily enjoying themselves! But one fine evening after twilight took its place by gently falling down over hillsides illuminating shadows casted all around town square where kids learned how some days can bring darkness too which wasn’t scary but adventurous instead – little mice whisper on grassy plain whilst watching stars above blink back at them; hedgehogs huddled together safe under thick thorn branches sharing stories while squirrel gathered nuts preparing lunches ahead knowing well enough those willpower moments spent wisely could lead him far beyond treetops soon…and oh so much more excitement awaited each creature now feeling less bored due to newfound freedom granted generously tonight alone.

In [27]:
# Inference original model
input_sentences = tokenizer("Schreibe eine kurze Gutenachtgeschichte für Kinder und benutze einfache Wörter.",
                            return_tensors="pt").to('cuda')

foundational_outputs_sentence = get_outputs(foundation_model,
                                            input_sentences,
                                            max_new_tokens=1000,
                                            )

print(tokenizer.batch_decode(foundational_outputs_sentence,
                             skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Schreibe eine kurze Gutenachtgeschichte für Kinder und benutze einfache Wörter. Ich möchte, dass Sie die Geschichte über Tiere handeln sollte!\nLiebe Kindern: Heute nochmal meines kleinen Geschichten aus der Welt unten am Himmel geht\'s um das Treibholz des Weihnachtenbaums ins Mädchenkissen bringenden Elch „Spike“, welcher in einem wildnisähnelndem Wald schlief; aber dann machte er einen Entschluss... Spannung ist ja auch bei den kleinsten lesenswerten Momentum vorhanden :) Und so viel Glück im Lesungen ;) ... Etwas anmarscht sicherlich nicht genauer hin zu "kompliziert". :-) Bitte füget mir Feedback mit Reizen oder Kritik wenn ihr was geben wollt - ich werde es gerne reingehen :)\n   Diese Geschichte dient eher zum Üben Sprachliche Sicherheit beziehungsweise Fremdsprachenlerner (oder Anbeter der deutschen Kunstsprichwort-Ecke), da man hier viele Verbindeteilende Merkmalen findet wie zB. Imagination bzw Erfolgsversprechende Aussagen – diese besitzt unsere Hauptfigur schon mal dabei 

Heute nochmal meines kleinen Geschichten aus der Welt unten am Himmel geht\'s um das Treibholz des Weihnachtenbaums ins Mädchenkissen bringenden Elch „Spike“, welcher in einem wildnisähnelndem Wald schlief; aber dann machte er einen Entschluss... Spannung ist ja auch bei den kleinsten lesenswerten Momentum vorhanden :) Und so viel Glück im Lesungen ;) ... Etwas anmarscht sicherlich nicht genauer hin zu "kompliziert". :-) Bitte füget mir Feedback mit Reizen oder Kritik wenn ihr was geben wollt - ich werde es gerne reingehen :)\n   Diese Geschichte dient eher zum Üben Sprachliche Sicherheit beziehungsweise Fremdsprachenlerner (oder Anbeter der deutschen Kunstsprichwort-Ecke), da man hier viele Verbindeteilende Merkmalen findet wie zB. Imagination bzw Erfolgsversprechende Aussagen – diese besitzt unsere Hauptfigur schon mal dabei als  Befindliches Aushauchen … Nennen wir das Zusammengesetztes allgemeine Princip / Lehrgegenstand einer solchen Art von Geschichte ‘Fabel’? Oder Variante davon unter Bezugnahme auf Puzzleteile sowie rdglosse Allusion zur Thematisierung natürlicher Relation zwischen spezieller Daseinsform/Personifiziaren Objecte(r) & Grundlagen Naturgesellschaftlichen Seiendevorgangs ? Das mag eigentlicht weniger herausnehmen können falls alle Details ausgearmblet werden.. DoT kommt spielerisch erst gar nöthig !!! So kannst Du doch selten etwas Einverständisses damit sondern du weilst immer wieder neu entdecnen !<br/>\nNun also los mit dem Abenteuer!!! <strong>"Aufbruchszeit"</Strong>: In dem traurigen Land unten betrieb Luna eine bahrrasse Bedienmannschaft gegenuber den Stromflüssigkeitshandwerkzeugspalette eines großen Fabrikantenkoncerniums kippt sein Gewicht vom Beinaheüberraschetend hervorgehensaftigen Standpunkt eines gewickelt vollbestehrend verharrensend geschaffener Rasterfensterbildners zu denen sehnlichem Bereitschaftszustande einem unaufhalting erschossenen Blätternfall durch blitzartiges Übertrittenserlebnisse.<strong> </STRONG>; Mit anderen aussagekräftigaeren Bestückten Formuliergängäsen sagede sie sich frühere Trubsal umsortieren konnte nachdem sie ihm den Schlussstrang aller Umgebahnhaltemechanismengewohnt worden hat…DARUMMUSSIETZWIELESCHONUNGENDEMAKENGLOBALLERJAGDBASISINDIVIDUALITATUNDSEINEPERSONALLIEBESINDERNÜBERGEMISTBEHOLDENSÄTTIGERSREIBELNOVAEMENTOFTHETAVELATIONISTIREREICHTWEITERANDRAUCOMBRUTISHOPRAFEMATICSFACTOSTONEGOODNESS..." Daß Gottlieger Herr Wercklemacher dieses Zeitalters nun lispelte seine Ohnmacht fortgemästigt haben will sag ich noch später, wenn ihn jegene Person selbst beruhrend heimwekkender Verpflegersinn zurücklassat in ihres Nachgangsamkeitssuchem stets regelmäßige Folgedoppelschrittprototypische Bewährertanzliniegamblingspiegelabgleichende Ausrichteteszenenskipperlaufbahnjägerbesitzerfunktionale Interfaceanpassungleistelektronicidynastsicallundingmaessensexperiences….

# Preparing the Dataset


In [ ]:
from datasets import load_dataset
dataset = "fka/awesome-chatgpt-prompts"

# Create the Dataset to create prompts.
data = load_dataset(dataset)

data = data.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
train_sample = data["train"].select(range(50))

del data
train_sample = train_sample.remove_columns('act')

display(train_sample)

In [ ]:
print(train_sample[:1])

# Finetuning

In [ ]:
# TARGET_MODULES
# https://github.com/huggingface/peft/blob/39ef2546d5d9b8f5f8a7016ec10657887a867041/src/peft/utils/other.py#L220

import peft
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16, #As bigger the R bigger the parameters to train.
    lora_alpha=16, # a scaling factor that adjusts the magnitude of the weight matrix. It seems that as higher more weight have the new training.
    target_modules=target_modules,
    lora_dropout=0.05, #Helps to avoid Overfitting.
    bias="none", # this specifies if the bias parameter should be trained.
    task_type="CAUSAL_LM"
)

In [ ]:
# Create a directory to contain the Model
import os
working_dir = './'

output_directory = os.path.join(working_dir, "peft_lab_outputs")

In [ ]:
# Creating the TrainingArgs
import transformers
from transformers import TrainingArguments # , Trainer
training_args = TrainingArguments(
    output_dir=output_directory,
    auto_find_batch_size=True, # Find a correct bvatch size that fits the size of Data.
    learning_rate= 2e-4, # Higher learning rate than full fine-tuning.
    num_train_epochs=5
)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
trainer = SFTTrainer(
    model=foundation_model,
    args=training_args,
    train_dataset=train_sample,
    peft_config = lora_config,
    dataset_text_field="prompt",
    tokenizer=tokenizer,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

In [ ]:
# Save the model.
peft_model_path = os.path.join(output_directory, f"lora_model")

In [ ]:
# Save the model.
trainer.model.save_pretrained(peft_model_path)

In [ ]:
# In case you are having memory problems uncomment this lines to free some memory
import gc
import torch
del foundation_model
del trainer
del train_sample
torch.cuda.empty_cache()
gc.collect()

# Inference with the pretrained model

In [ ]:
#import peft
from peft import AutoPeftModelForCausalLM, PeftConfig
#import os

device_map = {"": 0}
working_dir = './'

output_directory = os.path.join(working_dir, "peft_lab_outputs")
peft_model_path = os.path.join(output_directory, f"lora_model")

In [ ]:
bnb_config2 = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
#Load the Model.
loaded_model = AutoPeftModelForCausalLM.from_pretrained(
                                        peft_model_path,
                                        #torch_dtype=torch.bfloat16,
                                        is_trainable=False,
                                        #load_in_4bit=True,
                                        quantization_config=bnb_config2,
                                        device_map = 'cuda')

# Inference the fine-tuned model


In [ ]:
input_sentences = tokenizer("I want you to act as a motivational coach. ",
                            return_tensors="pt").to('cuda')
foundational_outputs_sentence = get_outputs(loaded_model,
                                            input_sentences,
                                            max_new_tokens=50)

print(tokenizer.batch_decode(foundational_outputs_sentence,
                             skip_special_tokens=True))